In [1]:
import sys
sys.path.append('..')
from utilities import configuration
from utilities import health_data

In [2]:
import json
config = configuration.get_config()

EXPERIMENT_CONFIGURATION_NAME='configuration_84'
experiment_configurations = json.load(open(config['experiments_config'], encoding='utf-8'))
X_train, y_train, X_test, y_test, features_names = health_data.Admission.get_train_test_matrices(experiment_configurations[EXPERIMENT_CONFIGURATION_NAME])
print(X_train.shape)

Enter key for decryption:  ········


 0 matrix.shape=(419139, 5)
 1 matrix.shape=(419139, 68)
 2 matrix.shape=(419139, 8011)
 3 matrix.shape=(419139, 5572)
(419139, 7500)


In [3]:
from sklearn.linear_model import LogisticRegression

In [4]:
MODEL_CONFIGURATION_NAME = 'model_302'
logreg = configuration.model_from_configuration_name(MODEL_CONFIGURATION_NAME)


In [5]:
print('Preparing to scale data')

Preparing to scale data


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.toarray())
X_test = scaler.transform(X_train.toarray())
print('Data scaled') 


In [ ]:
logreg.fit(X_train, 
           y_train)
print('Training finished')

In [19]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix


def _get_metric_evaluations(evaluated_model, X, y_true, model_config_name, description=''):
    y_pred = evaluated_model.predict(X)
    y_score = evaluated_model.predict_proba(X)[:,1]

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    results = {'Description': description,
                        'Precision': precision_score(y_true, y_pred),
                        'Recal': recall_score(y_true, y_pred),
                        'F1-Score': f1_score(y_true, y_pred),
                        'AUC': roc_auc_score(y_true=y_true, y_score=y_score),
                        'TN': tn,
                        'TP': tp,
                        'FN': fn,
                        'FP': fp,
                        'Model config': model_config_name
                        }
    results = {key: [results[key]] for key in results}
    return pd.DataFrame(results)

In [20]:
import pandas as pd
df = pd.concat([_get_metric_evaluations(logreg, X_train, y_train, MODEL_CONFIGURATION_NAME, description='LogReg train'),
                _get_metric_evaluations(logreg, X_test, y_test, MODEL_CONFIGURATION_NAME, description='LogReg test')])


df

,Description,Precision,Recal,F1-Score,AUC,TN,TP,FN,FP,Model config
0,LogReg train,0.084538,0.72032,0.151317,0.758386,265995,12535,4867,135742,model_302
0,LogReg test,0.080477,0.65321,0.143300,0.721272,66519,2961,1572,33832,model_302


In [28]:
diagnosis_mapping = health_data.Admission.get_diagnoses_mapping()
intervention_mapping = health_data.Admission.get_intervention_mapping()

In [31]:
def code2description(code):
    if code.upper() in diagnosis_mapping or code in diagnosis_mapping:
        assert not (code.upper() in intervention_mapping or code in intervention_mapping)
        return diagnosis_mapping[code.upper()]
    
    if code.upper() in intervention_mapping or code in intervention_mapping:
        assert not (code.upper() in diagnosis_mapping or code in diagnosis_mapping)
        return intervention_mapping[code.upper()]
    return "N/A"

In [33]:
import numpy as np
print('Finished computing coefficients (normalized features), formating and saving ...')
scored_feature_names = list(zip(list(logreg.coef_[0,:]),
                                features_names))

scored_feature_names = sorted(scored_feature_names, 
                              key=lambda x:np.abs(x[0]), reverse=True)

coefficients_df = pd.DataFrame(scored_feature_names,
                               columns=['Score', 'Feature Name'])

coefficients_df = coefficients_df[['Feature Name', 'Score']]
coefficients_df['Code Description'] = list(map(code2description, coefficients_df['Feature Name']))

# coefficients_df.to_csv(config['surrogate_logreg_coefficients'], index=False)
# print('DONE, saving metrics ...')
coefficients_df

Finished computing coefficients (normalized features), formating and saving ...


,Feature Name,Score,Code Description
0,i469,-3.636504,"{'Cardiac arrest, unspecified'}"
1,m161,-3.070155,"{'Other primary coxarthrosis', 'Coxarthrosis, ..."
2,c920,2.714981,{'Acute myeloblastic leukaemia [AML]'}
3,t84033,2.487516,"{'Mechanical complication of hip prosthesis, b..."
4,i634,-2.259758,"{'Cerebral infarction, unspecified', 'Cerebral..."
...,...,...,...
7495,d441,-0.000466,{'Neoplasm of uncertain or unknown behaviour o...
7496,1sq52ha,-0.000433,"{'Drainage, pelvis using percutaneous (needle)..."
7497,m7956,-0.000403,"{'Residual foreign body in soft tissue, lower ..."
7498,cmg,-0.000229,N/A


In [27]:
coefficients_df.iloc[:20,:]

,Feature Name,Score
0,i469,-3.636504
1,m161,-3.070155
2,c920,2.714981
3,t84033,2.487516
4,i634,-2.259758
5,c799,2.091978
6,g932,2.020501
7,d828,1.957170
8,5ac24ckbd,1.935947
9,1wa03hakc,1.860005
